In [2]:
MONKEYLEARN_CLASSIFIER_BASE_URL = 'https://api.monkeylearn.com/api/v1/categorizer/'
MONKEYLEARN_EXTRACTOR_BASE_URL = 'https://api.monkeylearn.com/api/v1/extraction/'

# This classifier is used to detect the tweet/bio's language
MONKEYLEARN_LANG_CLASSIFIER_ID = 'cl_hDDngsX8'

# This classifier is used to detect the tweet/bio's topics
MONKEYLEARN_TOPIC_CLASSIFIER_ID = 'cl_5icAVzKR'

# This extractor is used to extract keywords from tweets and bios
MONKEYLEARN_EXTRACTOR_ID = 'ex_y7BPYzNG'

In [ ]:
# tweepy is used to call the Twitter API from Python
import tweepy
import re
import credentials as cr
 
# Authenticate to Twitter API
auth = tweepy.OAuthHandler(cr.TWITTER_CONSUMER_KEY, cr.TWITTER_CONSUMER_SECRET)
auth.set_access_token(cr.TWITTER_ACCESS_TOKEN_KEY, cr.TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)